In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
pwd

'/Users/ChunyanHao/GitHub/DS_Takehome'

In [3]:
song = pd.read_json('/Users/ChunyanHao/desktop/github/DS_take_home/data/song.json')
song.head(5)

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [4]:
song.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 4000 non-null   object
 1   user_id            4000 non-null   int64 
 2   user_state         4000 non-null   object
 3   user_sign_up_date  4000 non-null   object
 4   song_played        4000 non-null   object
 5   time_played        4000 non-null   object
dtypes: int64(1), object(5)
memory usage: 187.6+ KB


In [5]:
song['user_sign_up_date'] = pd.to_datetime(song['user_sign_up_date'])
song['time_played'] = pd.to_datetime(song['time_played'])

In [6]:
for column in song.columns:
    uniques = song[column].unique()
    print('{0:20s} {1:5d}\t'.format(column, len(uniques)), uniques[:5])

id                    4000	 ['GOQMMKSQQH' 'HWKKBQKNWI' 'DKQSXVNJDH' 'HLHRIDQTUW' 'SUKJCSBCYW']
user_id                196	 [122   3  35 126   6]
user_state              41	 ['Louisiana' 'Ohio' 'New Jersey' 'Illinois' 'Texas']
user_sign_up_date       20	 ['2015-05-16T00:00:00.000000000' '2015-05-01T00:00:00.000000000'
 '2015-05-04T00:00:00.000000000' '2015-05-18T00:00:00.000000000'
 '2015-05-19T00:00:00.000000000']
song_played            100	 ['Hey Jude' 'We Can Work It Out' 'Back In the U.S.S.R.' 'P.s. I Love You'
 "Sgt. Pepper's Lonely Hearts Club Band"]
time_played           3997	 ['2015-06-11T21:51:35.000000000' '2015-06-06T16:49:19.000000000'
 '2015-06-14T02:11:29.000000000' '2015-06-08T12:26:10.000000000'
 '2015-06-28T14:57:00.000000000']


### 1. Question 1

What are the top 3 and the bottom 3 states in terms of number of users?

In [7]:
temp = song[['user_id', 'user_state']].drop_duplicates().groupby([
    'user_state']).count().sort_values(by = 'user_id', ascending = False)
temp.columns = ['user_count']
print('top 3: \n', temp.head(3), '\n', 'bottom 3: \n',temp.tail(3))

top 3: 
             user_count
user_state            
New York            23
California          21
Texas               15 
 bottom 3: 
             user_count
user_state            
Nebraska             1
Arizona              1
Kansas               1


### 2. Question 2

What are the top 3 and the bottom 3 states in terms of user engagement? 

You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little.

Because the CEO cares about "which states users are using the product a lot/very little", we can define the user engagement as: the average number of song each user played in the State.

In [8]:
temp = song[['user_state', 'user_id', 'song_played']].groupby(['user_state', 'user_id']).count().reset_index()
temp = temp[['user_state', 'song_played']].groupby(['user_state']).mean().reset_index()
temp.sort_values(by = 'song_played', ascending = False)[:4]

,user_state,song_played
22,Nebraska,36.000000
1,Alaska,29.000000
20,Mississippi,28.333333
33,South Carolina,28.333333


### 3. Question 3

The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Can you give him a list of those users?

In [9]:
temp = song[['user_id', 'user_state', 'user_sign_up_date']].drop_duplicates()

In [10]:
# ignore the tie when ranking by 'user_sign_up_date'
list_2 = temp.sort_values(by = 'user_sign_up_date').groupby(['user_state']).first().sort_values(by = 'user_sign_up_date')
list_2.reset_index()[:5]

,user_state,user_id,user_sign_up_date
0,Alabama,5,2015-05-01
1,Texas,7,2015-05-01
2,Oregon,1,2015-05-01
3,Ohio,3,2015-05-01
4,North Carolina,2,2015-05-01


In [11]:
# considering the tie when ranking by 'user_sign_up_date'
temp['rank'] = temp.groupby(['user_state'])['user_sign_up_date'].rank(method= 'dense')
list_2 = temp[temp['rank'] == 1].sort_values(by = ['user_sign_up_date','user_state']).reset_index()
list_2.iloc[:5,1:-1]

,user_id,user_state,user_sign_up_date
0,5,Alabama,2015-05-01
1,6,New Jersey,2015-05-01
2,4,New Mexico,2015-05-01
3,2,North Carolina,2015-05-01
4,3,Ohio,2015-05-01


### 4 Question 4

Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. 

That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.

There should be a lot of different methods for this question: 

1) The simplest method is to choose the most popular songs. 

2) In addition, k-nearest neighbor (KNN) method can also be used. 

3) More advancely, collaborative filtering method can be implemented.

#### 4.1 choose the most popular songs

In [12]:
temp = song[['song_played', 'user_id']].groupby(['song_played']).count()
temp
temp.columns = ['count']
song_rank = temp.sort_values(by = 'count', ascending = False)
song_rank['played'] = False

In [13]:
def song_recommendation_v1(current_song, song_rank):
    song_rank.at[current_song, 'played'] = True
    for i in song_rank.index:
        if song_rank.at[i, 'played'] == False:
            song_rank.at[i, 'played'] = True
            return i

In [14]:
v1_song_rank = song_rank.copy()
song_recommendation_v1('Get Back', v1_song_rank)

'Come Together'

#### 4.2  k-nearest neighbor (KNN) 

In [15]:
temp = song[['user_id', 'song_played', 'time_played']].groupby(['user_id', 'song_played']).count().reset_index()
pivot_df = temp.pivot_table(index = 'song_played', columns = 'user_id', values = 'time_played').fillna(0)
pivot_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0.0,0.0,1.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,3.0,0.0,2.0,0.0,0.0,2.0,0.0
A Hard Day's Night,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
A Saturday Club Xmas/Crimble Medley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ANYTIME AT ALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Across The Universe,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [17]:
pivot_matrix = csr_matrix(pivot_df.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(pivot_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
pivot_df.shape

(100, 196)

In [19]:
query_index = np.random.choice(pivot_df.shape[0])
print(query_index)
distance, indices = model_knn.kneighbors(pivot_df.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 5)

49


In [20]:
for i in range(0, len(distance.flatten())):
    if i == 0:
        print('recommendations for {0}:\n'.format(pivot_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, 
                                                       pivot_df.index[indices.flatten()[i]],
                                                       distance.flatten()[i]))

recommendations for LITTLE CHILD:

1: Across The Universe, with distance of 0.7763932022500211:
2: I'm A Loser, with distance of 0.7958758547680684:
3: ANYTIME AT ALL, with distance of 0.8110177634953863:
4: Everybody's Got Something To Hide Except Me And My Monk, with distance of 0.8110177634953863:


In [21]:
def song_recommendation_v2(song, song_name, num_recommend):
    temp = song[['user_id', 'song_played', 'time_played']].groupby(['user_id', 'song_played']).count().reset_index()
    pivot_df = temp.pivot_table(index='song_played', columns='user_id', values='time_played').fillna(0)
    pivot_matrix = csr_matrix(pivot_df.values)
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors = num_recommend+1)
    model_knn.fit(pivot_matrix)
    distance, indices = model_knn.kneighbors(pivot_df.loc[song_name,].values.reshape(1, -1), n_neighbors = num_recommend+1)
    
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('recommendations for {0}:\n'.format(song_name))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, 
                                                           pivot_df.index[indices.flatten()[i]],
                                                           distance.flatten()[i]))

In [22]:
song_recommendation_v2(song, 'Revolution', 3)

recommendations for Revolution:

1: Come Together, with distance of 0.31919336124487685:
2: Get Back, with distance of 0.33962572516686096:
3: While My Guitar Gently Weeps, with distance of 0.36794900465266966:


#### 4.3  Collaborative filtering 

In [23]:
temp = song[['user_id', 'song_played', 'time_played']].groupby(['user_id', 'song_played']).count().reset_index()
pivot_df = temp.pivot_table(index = 'song_played', columns = 'user_id', values = 'time_played').fillna(0)
pivot_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0.0,0.0,1.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,3.0,0.0,2.0,0.0,0.0,2.0,0.0
A Hard Day's Night,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
A Saturday Club Xmas/Crimble Medley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ANYTIME AT ALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Across The Universe,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
song_user = song.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user = (song_user > 0).astype(int)

song_user.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,1,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.preprocessing import normalize
# Step 2: build song-song similarity matrix
song_user_norm = normalize(song_user, axis=1)  # normalize the song-user matrix
similarity = np.dot(song_user_norm, song_user_norm.T)  # calculate the similarity matrix
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

similarity_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.264392,0.139347,0.148968,0.132196,0.301023,0.295599,0.098533,0.197066,0.201129,...,0.516528,0.056888,0.578459,0.279852,0.399723,0.088131,0.330489,0.365433,0.164222,0.0
A Hard Day's Night,0.264392,1.000000,0.000000,0.000000,0.100000,0.146385,0.111803,0.000000,0.000000,0.091287,...,0.305788,0.129099,0.266996,0.000000,0.000000,0.000000,0.050000,0.215003,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.139347,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.161165,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.148968,0.000000,0.000000,1.000000,0.000000,0.164957,0.094491,0.125988,0.000000,0.000000,...,0.172292,0.000000,0.188044,0.097590,0.191663,0.000000,0.000000,0.103835,0.000000,0.0
Across The Universe,0.132196,0.100000,0.000000,0.000000,1.000000,0.097590,0.000000,0.000000,0.000000,0.000000,...,0.101929,0.000000,0.133498,0.000000,0.000000,0.000000,0.000000,0.061430,0.000000,0.0


In [26]:
def song_recommendation_v3(song, similarity, k):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    
    return df

In [27]:
song_recommendation_v3('Revolution', similarity_df, 3)

,Song,Similarity
0,Come Together,0.790756
1,Get Back,0.781407
2,Let It Be,0.725972
